In [1]:
import findspark
findspark.init('/home/osboxes/spark-2.4.3-bin-hadoop2.7')
import pyspark
import os

In [4]:
myPath = os.path.join('/home', 'osboxes', 'CourseMaterial', 'Spark_for_Machine_Learning', 'Logistic_Regression')

print(myPath)

os.chdir(myPath)

/home/osboxes/CourseMaterial/Spark_for_Machine_Learning/Logistic_Regression


In [5]:
os.listdir()

['Titanic_Log_Regression_Code_Along.ipynb',
 'new_customers.csv',
 '.ipynb_checkpoints',
 'titanic.csv',
 'sample_libsvm_data.txt',
 'Logistic_Regression_Consulting_Project_SOLUTIONS.ipynb',
 'Logistic_Regression_Example.ipynb',
 'Logistic_Regression_Consulting_Project.ipynb',
 'customer_churn.csv']

In [6]:
# set up the session info

from pyspark.sql import SparkSession

name = 'log_reg'

spark = SparkSession.builder.appName(name).getOrCreate()

from pyspark.ml.classification import LogisticRegression

In [7]:
my_data = spark.read.format('libsvm').load('sample_libsvm_data.txt')

In [8]:
my_data.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(692,[127,128,129...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[124,125,126...|
|  1.0|(692,[152,153,154...|
|  1.0|(692,[151,152,153...|
|  0.0|(692,[129,130,131...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[99,100,101,...|
|  0.0|(692,[154,155,156...|
|  0.0|(692,[127,128,129...|
|  1.0|(692,[154,155,156...|
|  0.0|(692,[153,154,155...|
|  0.0|(692,[151,152,153...|
|  1.0|(692,[129,130,131...|
|  0.0|(692,[154,155,156...|
|  1.0|(692,[150,151,152...|
|  0.0|(692,[124,125,126...|
|  0.0|(692,[152,153,154...|
|  1.0|(692,[97,98,99,12...|
|  1.0|(692,[124,125,126...|
+-----+--------------------+
only showing top 20 rows



In [9]:
logReg = LogisticRegression()

In [10]:
logReg_fit = logReg.fit(my_data)

In [11]:
log_summary = logReg_fit.summary

In [12]:
# check whether label  matches the predictions

log_summary.predictions.printSchema()

root
 |-- label: double (nullable = true)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



In [13]:
log_summary.predictions.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(692,[127,128,129...|[19.8534775947478...|[0.99999999761359...|       0.0|
|  1.0|(692,[158,159,160...|[-20.377398194908...|[1.41321555111056...|       1.0|
|  1.0|(692,[124,125,126...|[-27.401459284891...|[1.25804865126979...|       1.0|
|  1.0|(692,[152,153,154...|[-18.862741612668...|[6.42710509170303...|       1.0|
|  1.0|(692,[151,152,153...|[-20.483011833009...|[1.27157209200604...|       1.0|
|  0.0|(692,[129,130,131...|[19.8506078990277...|[0.99999999760673...|       0.0|
|  1.0|(692,[158,159,160...|[-20.337256674833...|[1.47109814695581...|       1.0|
|  1.0|(692,[99,100,101,...|[-19.595579753418...|[3.08850168102631...|       1.0|
|  0.0|(692,[154,155,156...|[19.2708803215613...|[0.99999999572670...|       0.0|
|  0.0|(692,[127

In [14]:
# use a tuple for train test

train, test = my_data.randomSplit([0.7, 0.3])

In [15]:
# fit on the training data

final_model = LogisticRegression()

fit_final = final_model.fit(train)

prediction_labels = fit_final.evaluate(test)

In [16]:
prediction_labels.predictions.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(692,[100,101,102...|[9.95459240688709...|[0.99995249330524...|       0.0|
|  0.0|(692,[124,125,126...|[28.7599116957902...|[0.99999999999967...|       0.0|
|  0.0|(692,[124,125,126...|[30.3138789808392...|[0.99999999999993...|       0.0|
|  0.0|(692,[127,128,129...|[23.2148001300700...|[0.99999999991721...|       0.0|
|  0.0|(692,[152,153,154...|[31.2292957072338...|[0.99999999999997...|       0.0|
|  0.0|(692,[152,153,154...|[15.1175142571215...|[0.99999972801380...|       0.0|
|  0.0|(692,[155,156,180...|[34.4419417702344...|[0.99999999999999...|       0.0|
|  1.0|(692,[119,120,121...|[-2.4110013893119...|[0.08233762336249...|       1.0|
|  1.0|(692,[123,124,125...|[-23.547565622499...|[5.93502001471869...|       1.0|
|  1.0|(692,[123

In [17]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator


In [18]:
# set up the evaluator

my_eval = BinaryClassificationEvaluator()

roc_results = my_eval.evaluate(prediction_labels.predictions)

roc_results

1.0